In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
#statsDF - initial stats dataframe
#gamesDF - initial games dataframe
#newGamesDF - formatted games dataframe with game date as index and rest of record in one row
#finalStats - formatted stats dataframe with player's name as the index
#mainDF: The dataframe where the main analysis will take place.

In [258]:
#Dump the stats dataframe into a csv and clean it up
cols = ['PLAYER', 'MIN', 'FGM','FGA', 'FG%', '3PM', '3PA','3P%','FTM','FTA','FT%','OREB','DREB','REB','AST','TOV','STL','BLK','PF','PTS','+/-']
statsDF = pd.read_csv('stats.csv', header= None, names=cols, index_col=False, error_bad_lines=False, delim_whitespace=True)
statsDF[cols] = statsDF[cols].replace({'\$': '', ',': ''}, regex=True)

In [259]:
statsDF = statsDF[statsDF.PLAYER != 'PLAYER']
statsDF = statsDF[statsDF.PLAYER != 'Totals:']
statsDF = statsDF[statsDF.PLAYER != 'INACTIVE']
statsDF = statsDF.reset_index(drop=True)

In [260]:
home_city = []
flag = 0
away_player_count = 0
for index,row in statsDF.iterrows():
    if row[0] == 'DNP':
        flag = 1
    else:
        if statsDF.iloc[index + 1][0] == 'DNP':
            continue
        elif flag == 1:
            home_city.append(row[0])
            if index % 2 == 0:
                home_city.append(statsDF.iloc[-1][0])
                away_city = [statsDF.iloc[0][0],statsDF.iloc[-2][0]]
                statsDF = statsDF[statsDF.PLAYER != home_city[1]]
                statsDF = statsDF[statsDF.PLAYER != away_city[0]]
                statsDF = statsDF[statsDF.PLAYER != away_city[1]]
                statsDF = statsDF.reset_index(drop=True)
                away_player_count = statsDF.index[index] / 2
                break
            else:
                home_city.append(statsDF.iloc[-1][0])
                away_city = [statsDF.iloc[0][0],statsDF.iloc[-2][0]]
                statsDF = statsDF[statsDF.PLAYER != home_city[0]]
                statsDF = statsDF[statsDF.PLAYER != home_city[1]]
                statsDF = statsDF[statsDF.PLAYER != away_city[0]]
                statsDF = statsDF[statsDF.PLAYER != away_city[1]]
                statsDF = statsDF.reset_index(drop=True)
                away_player_count = int(statsDF.index[index] / 2)
                break
        else:
            continue


In [261]:
#Setting up the index on the main stats dataframe
mainDFIndex = []
for index, row in statsDF.iterrows():
    if index % 2 == 0:
        mainDFIndex.append(str(row['PLAYER']) + " " + str(row['MIN']))
print(mainDFIndex)

['Gordon Hayward', 'Jayson Tatum', 'Al Horford', 'Jaylen Brown', 'Kyrie Irving', 'Marcus Smart', 'Aron Baynes', 'Semi Ojeleye', 'Terry Rozier', 'Shane Larkin', 'Abdel Nader', 'Daniel Theis', 'LeBron James', 'Jae Crowder', 'Kevin Love', 'Dwyane Wade', 'Derrick Rose', 'Tristan Thompson', 'JR Smith', 'Iman Shumpert', 'Jeff Green', 'Kyle Korver', 'Jose Calderon', 'Channing Frye', 'Cedi Osman']


In [280]:
#Setup dicts with the sub categories
newDict = dict()
something = statsDF.T.to_dict().values()
pos = ''
for index, i in enumerate(something):
    if index % 2 != 0:
        newDict[index] = i
        newDict[index]['POS'] = pos
    else:
        if i['FGM'] == 'nan':
            continue
        else:
            pos = i['FGM']


finalStats = pd.DataFrame.from_dict(newDict,orient='index')
finalStats.index = mainDFIndex
finalStats.drop(['+/-'], axis=1, inplace=True)
finalStats = finalStats.rename(columns={'PLAYER':'MIN','MIN':'FGM', 'FGM':'FGA', 'FGA':'FG%', 'FG%':'3PM', '3PM':'3PA', '3PA':'3P%', '3P%':'FTM', 'FTM':'FTA', 'FTA':'FT%', 'FT%':'OREB', 'OREB':'DREB', 'DREB':'REB', 'REB':'AST', 'AST':'TOV', 'TOV':'BLK', 'BLK':'PF', 'PF':'PTS', 'PTS':'+/-'})
player_count = len(finalStats.index)
teams_list = ((away_city[0] + " ") * away_player_count) + ((home_city[0] + " ") * (player_count - away_player_count))
teams_list = teams_list.split()
opponent_list = teams_list[::-1]
finalStats['TEAM'] = teams_list
finalStats['OPPONENT'] = opponent_list
#This will replace the nan's with DNP
finalStats = finalStats.fillna('NA')

In [282]:
finalStats

,FGA,PTS,FT%,+/-,TOV,3P%,FG%,POS,FTA,MIN,...,OREB,FTM,AST,PF,STL,FGM,DREB,BLK,TEAM,OPPONENT
Gordon Hayward,2,2,0.0,3,0,0.0,50.0,F,0,5:15,...,0,0,0,1,0,1,1,0,Boston,Cleveland
Jayson Tatum,12,14,100,6,1,50.0,41.7,F,3,36:32,...,4,3,3,4,0,5,6,0,Boston,Cleveland
Al Horford,7,9,71.4,8,0,0.0,28.6,C,7,32:07,...,0,5,5,2,1,2,7,0,Boston,Cleveland
Jaylen Brown,23,25,50.0,-5,3,22.2,47.8,G,2,39:36,...,1,1,0,5,0,11,5,2,Boston,Cleveland
Kyrie Irving,17,22,100,-1,2,44.4,47.1,G,2,39:21,...,2,2,10,4,0,8,2,3,Boston,Cleveland
Marcus Smart,16,12,66.7,-8,2,0.0,31.3,NA,3,35:03,...,0,2,3,2,2,5,9,2,Boston,Cleveland
Aron Baynes,2,6,50.0,-14,2,0.0,100,NA,4,19:06,...,2,2,1,5,1,2,3,0,Boston,Cleveland
Semi Ojeleye,2,0,0.0,-10,0,0.0,0.0,NA,0,8:39,...,0,0,0,1,0,0,0,0,Boston,Cleveland
Terry Rozier,6,9,100,4,0,33.3,33.3,NA,4,19:32,...,0,4,2,0,0,2,3,4,Boston,Cleveland
Shane Larkin,1,0,0.0,2,0,0.0,0.0,NA,0,4:49,...,0,0,0,0,0,0,1,0,Boston,Cleveland


In [331]:
for index,row in finalStats.iterrows():
    finalStats['Date'] = newGamesDF['Date'][0]
    finalStats['1st Qtr H'] = newGamesDF['1st Qtr H'][0]
    finalStats['2nd Qtr H'] = newGamesDF['2nd Qtr H'][0]
    finalStats['3rd Qtr H'] = newGamesDF['3rd Qtr H'][0]
    finalStats['4th Qtr H'] = newGamesDF['4th Qtr H'][0]
    finalStats['1st Qtr A'] = newGamesDF['1st Qtr A'][0]
    finalStats['2nd Qtr A'] = newGamesDF['2nd Qtr A'][0]
    finalStats['3rd Qtr A'] = newGamesDF['3rd Qtr A'][0]
    finalStats['4th Qtr A'] = newGamesDF['4th Qtr A'][0]
    finalStats['Total H'] = newGamesDF['Total H'][0]
    finalStats['Total A'] = newGamesDF['Total A'][0]
    finalStats['Ref1'] = newGamesDF['Ref1'][0]
    finalStats['Ref2'] = newGamesDF['Ref2'][0]
    finalStats['Ref3'] = newGamesDF['Ref3'][0]
    finalStats['W/L'] = newGamesDF['W/L'][0]
    finalStats['Home'] = newGamesDF['Home'][0]
    finalStats['Away'] = newGamesDF['Away'][0]
    

In [334]:
#This function will calculate the projected fantasy points per game
def fantasy_points(row):
    try:
        score = (int(row['3PM']) * 3) + (int(row['FGM']) * 2) + (int(row['REB']) * 1) + (int(row['AST']) * 1.5) + (int(row['BLK']) * 3) + (int(row['STL']) * 3) + (int(row['TOV']) * -1)
        return score
    except:
        pass

finalStats['Fantasy Score'] = finalStats.apply(fantasy_points, axis=1)

In [336]:
finalStats['Fantasy Score']

Gordon Hayward       3.0
Jayson Tatum        26.5
Al Horford          21.5
Jaylen Brown        37.0
Kyrie Irving        54.0
Marcus Smart        33.5
Aron Baynes         11.5
Semi Ojeleye         0.0
Terry Rozier        25.0
Shane Larkin         1.0
Abdel Nader          NaN
Daniel Theis         NaN
LeBron James        58.5
Jae Crowder         22.0
Kevin Love          20.0
Dwyane Wade         14.5
Derrick Rose        18.0
Tristan Thompson    11.0
JR Smith            16.5
Iman Shumpert        8.0
Jeff Green           5.0
Kyle Korver          0.0
Jose Calderon        NaN
Channing Frye        NaN
Cedi Osman           NaN
Name: Fantasy Score, dtype: float64